In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# CSP

**Sources:**
 
 - https://www.cs.ubc.ca/~mack/CS322/lectures/3-CSP2.pdf

**Definition:** A _constraint satisfaction problem (CSP)_ consists of:

 * a set of variables $\mathscr V$.
 * a domain $\textrm{dom}(V)$ for each variable $V \in \mathscr V$.
 * a set of constraints $C$.
 
An example of a CSP model is:

 * $\mathscr V = \{V_1, V_2\}$
   * $\textrm{dom}(V_1) = \{1,2,3\}$
   * $\textrm{dom}(V_2) = \{1,2\}$
 * $C = \{C_1,C_2,C_3\}$
   * $C_1: V_2 \neq 2$
   * $C_2: V_1 + V_2 < 5$
   * $C_3: V_1 > V_2$
 
**Definition**: A _model_ of a CSP is an assignment of values to all of its variables that _satisfies_ all of its constraints.

**Generate and Test (GT) algorithm**: Systematically check all possible worlds. All possible worlds is the cross product of all the domains:

$$ \textrm{dom}(V_1) \times \textrm{dom}(V_2) \times \ldots \times \textrm{dom}(V_n) $$

Generate and test:

 1. Generate possible worlds one at a time.
 2. Test constraints for each one.
 
For $k$ variables, each with domain size $d$, and there are $c$ constraints, the complexity is $O(ck^d)$:

 * There are $d^k$ possible worlds.
 * For each one need to check c constraints.

# Implementation

**CSP solver implementation:**

In [501]:
import copy

def head(xs): return xs[0] if len(xs) > 0 else None
def replace_all(text, dic):
    for k, v in dic.items():
        text = text.replace(k, str(v))
    return text
def subset(A,B): return all([a in B for a in A])

class Variable():
    def __init__(self, name, D):
        self.name = name
        self.domain = D
        self.value = D[0]
    
    def __eq__(self, v): return v.name == self.name
    def __repr__(self): return '{}={}'.format(self.name, self.value, self.domain)
        
class Constraint():
    def __init__(self, expr, *variables):
        self.variables = variables
        self.expression = expr
    def evalstr(self): return 
    def eval(self, V): 
        return eval(replace_all(self.expression, {v.name:v.value for v in V}))
    def __repr__(self): return self.expression
    
class CSP():
    def __init__(self, variables, constraints):
        self.variables = variables
        self.constraints = constraints
        self.stop_on_first = False
        
    def solve(self):
        self.solution = []
        self.gt_solve([], self.variables)
        return self.solution[::-1]
    
    # solving with a Generate and Test (GT) algorithm
    #
    # example:
    # for a \in dom A
    #   for b \in dom B
    #      for c \in dom C
    #        if {A=a, B=b, C=c} return solution
    def gt_solve(self, S, V):
        print('(Call) S contains {}, V contains {}'.format(S,V))
        if len(V) == 0:
            
            # -- eval all
            #return all([c.eval(S) for c in self.constraints])
            
            # -- verbose output
            print('  (Base) Checking constraints...')
            for c in self.constraints:
                r = c.eval(S)
                if r: print('  (Satisfied) {}'.format(c))
                else: 
                    print('  (Failed) {}'.format(c))
                    return False
            return True
            
        v = V.pop()
        S.append(v)
        for d in v.domain:
            if self.solution and self.stop_on_first: return
            v.value = d
            if self.gt_solve(copy.deepcopy(S), copy.deepcopy(V)):
                self.solution = S
                print('  (Solution) {}'.format(S[::-1]))
                return
        return False

## Example 1

Testing the `gt_solve` method with the example model shown at the beginning of this document:

In [476]:
v1 = Variable('v1', [1,2,3])
v2 = Variable('v2', [1,2])
V = [v1,v2]
c1 = Constraint('v1 != v2', v1, v2)
c2 = Constraint('v1 + v2 <= 5', v1, v2)
c3 = Constraint('v1 > v2', v1, v2)
c4 = Constraint('v1 >= 3', v1)
C = [c1,c2,c3,c4]   
csp = CSP(V, C)
csp.stop_on_first = True
csp.solve()

(Call) S contains [], V contains [v1=1, v2=1]
(Call) S contains [v2=1], V contains [v1=1]
(Call) S contains [v2=1, v1=1], V contains []
  (Base) Checking constraints...
  (Failed) v1 != v2
(Call) S contains [v2=1, v1=2], V contains []
  (Base) Checking constraints...
  (Satisfied) v1 != v2
  (Satisfied) v1 + v2 <= 5
  (Satisfied) v1 > v2
  (Failed) v1 >= 3
(Call) S contains [v2=1, v1=3], V contains []
  (Base) Checking constraints...
  (Satisfied) v1 != v2
  (Satisfied) v1 + v2 <= 5
  (Satisfied) v1 > v2
  (Satisfied) v1 >= 3
  (Solution) [v1=3, v2=1]


[v1=3, v2=1]

## Example 2

Solving the Australian map coloring problem.

In [471]:
%%time
colors = {'red': 0, 'blue': 1, 'green': 2}
icolors = {v:k for k,v in colors.items()}
V = [Variable(x, list(colors.values())) for x in 'SA,WA,NT,Q,NSW,V'.split(',')]
V
c1 = Constraint('SA!=WA')
c2 = Constraint('SA!=NT')
c3 = Constraint('SA!=Q')
c4 = Constraint('SA!=NSW')
c5 = Constraint('SA!=V')
c6 = Constraint('WA!=NT')
c7 = Constraint('NT!=Q')
c8 = Constraint('Q!=NSW')
c9 = Constraint('NSW!=V')
C = [c1,c2,c3,c4,c5,c6,c7,c8,c9]
csp = CSP(V,C)
csp.stop_on_first = True
csp.solve()

(Call) S contains [], V contains [SA=0, WA=0, NT=0, Q=0, NSW=0, V=0]
(Call) S contains [V=0], V contains [SA=0, WA=0, NT=0, Q=0, NSW=0]
(Call) S contains [V=0, NSW=0], V contains [SA=0, WA=0, NT=0, Q=0]
(Call) S contains [V=0, NSW=0, Q=0], V contains [SA=0, WA=0, NT=0]
(Call) S contains [V=0, NSW=0, Q=0, NT=0], V contains [SA=0, WA=0]
(Call) S contains [V=0, NSW=0, Q=0, NT=0, WA=0], V contains [SA=0]
(Call) S contains [V=0, NSW=0, Q=0, NT=0, WA=0, SA=0], V contains []
  (Base) Checking constraints...
  (Failed) SA!=WA
(Call) S contains [V=0, NSW=0, Q=0, NT=0, WA=0, SA=1], V contains []
  (Base) Checking constraints...
  (Satisfied) SA!=WA
  (Satisfied) SA!=NT
  (Satisfied) SA!=Q
  (Satisfied) SA!=NSW
  (Satisfied) SA!=V
  (Failed) WA!=NT
(Call) S contains [V=0, NSW=0, Q=0, NT=0, WA=0, SA=2], V contains []
  (Base) Checking constraints...
  (Satisfied) SA!=WA
  (Satisfied) SA!=NT
  (Satisfied) SA!=Q
  (Satisfied) SA!=NSW
  (Satisfied) SA!=V
  (Failed) WA!=NT
(Call) S contains [V=0, NSW=

In [468]:
[{v.name:icolors[v.value]} for v in csp.solution]

[{'V': 'red'},
 {'NSW': 'blue'},
 {'Q': 'red'},
 {'NT': 'blue'},
 {'WA': 'red'},
 {'SA': 'green'}]

# Results

This is a concise implementation that solves CSP models with the `gt_solve` algorithm.

In [543]:
import copy

class Variable():
    def __init__(self, name, D): 
        self.name = name; 
        self.domain = D; 
        self.value = D[0]
    def __eq__(self, v): return v.name == self.name
    def __repr__(self): return '{}={}'.format(self.name, self.value, self.domain)
        
class Constraint():
    def __init__(self, expr): self.expression = expr
    def __repr__(self): return self.expression
    def eval(self, V): return eval(self.replace_all(self.expression, {v.name:v.value for v in V}))
    def replace_all(self, text, dic):
        for k, v in dic.items():
            text = text.replace(k, str(v))
        return text
    
class CSP():
    def __init__(self, variables, constraints):
        self.variables = variables
        self.constraints = constraints
        self.stop_on_first = False
        
    def solve(self):
        self.solution = []
        self.gt_solve([], self.variables)
        return self.solution[::-1]
    
    def gt_solve(self, S, V):
        if len(V) == 0: return all([c.eval(S) for c in self.constraints])           
        v = V.pop()
        S.append(v)
        for d in v.domain:
            if self.solution and self.stop_on_first: return
            v.value = d
            if self.gt_solve(copy.deepcopy(S), copy.deepcopy(V)):
                self.solution.append(S)
        return False

Applying this to the map coloring problem:

In [544]:
%%time
colors = {'red': 0, 'blue': 1, 'green': 2}
V = [Variable(x, list(colors.values())) for x in 'SA,WA,NT,Q,NSW,V'.split(',')]
c1 = Constraint('SA!=WA')
c2 = Constraint('SA!=NT')
c3 = Constraint('SA!=Q')
c4 = Constraint('SA!=NSW')
c5 = Constraint('SA!=V')
c6 = Constraint('WA!=NT')
c7 = Constraint('NT!=Q')
c8 = Constraint('Q!=NSW')
c9 = Constraint('NSW!=V')
C = [c1,c2,c3,c4,c5,c6,c7,c8,c9]
csp = CSP(V,C)
csp.solve()
print('There are {} solutions:'.format(len(csp.solution)))
lkup = {v:k for k,v in colors.items()}
print([[{v.name:lkup[v.value]} for v in s] for s in csp.solution])

There are 6 solutions:
[[{'V': 'red'}, {'NSW': 'blue'}, {'Q': 'red'}, {'NT': 'blue'}, {'WA': 'red'}, {'SA': 'green'}], [{'V': 'red'}, {'NSW': 'green'}, {'Q': 'red'}, {'NT': 'green'}, {'WA': 'red'}, {'SA': 'green'}], [{'V': 'blue'}, {'NSW': 'red'}, {'Q': 'blue'}, {'NT': 'red'}, {'WA': 'blue'}, {'SA': 'green'}], [{'V': 'blue'}, {'NSW': 'green'}, {'Q': 'blue'}, {'NT': 'green'}, {'WA': 'blue'}, {'SA': 'green'}], [{'V': 'green'}, {'NSW': 'red'}, {'Q': 'green'}, {'NT': 'red'}, {'WA': 'green'}, {'SA': 'green'}], [{'V': 'green'}, {'NSW': 'blue'}, {'Q': 'green'}, {'NT': 'blue'}, {'WA': 'green'}, {'SA': 'green'}]]
Wall time: 210 ms
